Practice from edX course: UCSanDiegoX: DSE200x

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [4]:
data = pd.read_csv("\\weather\\daily_weather.csv")
data.shape

(1095, 11)

In [5]:
data.head()

,number,air_pressure_9am,air_temp_9am,avg_wind_direction_9am,avg_wind_speed_9am,max_wind_direction_9am,max_wind_speed_9am,rain_accumulation_9am,rain_duration_9am,relative_humidity_9am,relative_humidity_3pm
0,0,918.060000,74.822000,271.100000,2.080354,295.400000,2.863283,0.0,0.0,42.420000,36.160000
1,1,917.347688,71.403843,101.935179,2.443009,140.471548,3.533324,0.0,0.0,24.328697,19.426597
2,2,923.040000,60.638000,51.000000,17.067852,63.700000,22.100967,0.0,20.0,8.900000,14.460000
3,3,920.502751,70.138895,198.832133,4.337363,211.203341,5.190045,0.0,0.0,12.189102,12.742547
4,4,921.160000,44.294000,277.800000,1.856660,136.500000,2.863283,8.9,14730.0,92.410000,76.740000


In [6]:
data.columns

Index(['number', 'air_pressure_9am', 'air_temp_9am', 'avg_wind_direction_9am',
       'avg_wind_speed_9am', 'max_wind_direction_9am', 'max_wind_speed_9am',
       'rain_accumulation_9am', 'rain_duration_9am', 'relative_humidity_9am',
       'relative_humidity_3pm'],
      dtype='object')

In [14]:
data.isnull().any(axis = 0)

air_pressure_9am          False
air_temp_9am              False
avg_wind_direction_9am    False
avg_wind_speed_9am        False
max_wind_direction_9am    False
max_wind_speed_9am        False
rain_accumulation_9am     False
rain_duration_9am         False
relative_humidity_9am     False
relative_humidity_3pm     False
dtype: bool

In [8]:
del data['number']

number column was redundant so just delete it.

In [9]:
data = data.dropna()
data.head()

,air_pressure_9am,air_temp_9am,avg_wind_direction_9am,avg_wind_speed_9am,max_wind_direction_9am,max_wind_speed_9am,rain_accumulation_9am,rain_duration_9am,relative_humidity_9am,relative_humidity_3pm
0,918.060000,74.822000,271.100000,2.080354,295.400000,2.863283,0.0,0.0,42.420000,36.160000
1,917.347688,71.403843,101.935179,2.443009,140.471548,3.533324,0.0,0.0,24.328697,19.426597
2,923.040000,60.638000,51.000000,17.067852,63.700000,22.100967,0.0,20.0,8.900000,14.460000
3,920.502751,70.138895,198.832133,4.337363,211.203341,5.190045,0.0,0.0,12.189102,12.742547
4,921.160000,44.294000,277.800000,1.856660,136.500000,2.863283,8.9,14730.0,92.410000,76.740000


In [16]:
data.shape

(1064, 10)

Make a copy of the data that has been cleaned.

In [17]:
cleanData = data.copy()

We are looking to predict the relative humidity at 3pm using the weather data we have for 9am. Thus, we are binarizing the 3pm humidity data so that any humidity above 24.99% is considered to be high. We can then predict when the humidity might be considered high.

In [19]:
cleanData['high_humidity_label'] = (cleanData['relative_humidity_3pm'] > 24.99)*1
print(cleanData['high_humidity_label'])

0       1
1       0
2       0
3       0
4       1
       ..
1090    1
1091    1
1092    1
1093    1
1094    0
Name: high_humidity_label, Length: 1064, dtype: int32


y is the target data and x will be all of the morning weather data.

In [20]:
y = cleanData[['high_humidity_label']].copy()

In [21]:
cleanData.head()

,air_pressure_9am,air_temp_9am,avg_wind_direction_9am,avg_wind_speed_9am,max_wind_direction_9am,max_wind_speed_9am,rain_accumulation_9am,rain_duration_9am,relative_humidity_9am,relative_humidity_3pm,high_humidity_label
0,918.060000,74.822000,271.100000,2.080354,295.400000,2.863283,0.0,0.0,42.420000,36.160000,1
1,917.347688,71.403843,101.935179,2.443009,140.471548,3.533324,0.0,0.0,24.328697,19.426597,0
2,923.040000,60.638000,51.000000,17.067852,63.700000,22.100967,0.0,20.0,8.900000,14.460000,0
3,920.502751,70.138895,198.832133,4.337363,211.203341,5.190045,0.0,0.0,12.189102,12.742547,0
4,921.160000,44.294000,277.800000,1.856660,136.500000,2.863283,8.9,14730.0,92.410000,76.740000,1


In [22]:
y.head()

,high_humidity_label
0,1
1,0
2,0
3,0
4,1


In [23]:
morningFeatures = ['air_pressure_9am','air_temp_9am','avg_wind_direction_9am','avg_wind_speed_9am',
        'max_wind_direction_9am','max_wind_speed_9am','rain_accumulation_9am',
        'rain_duration_9am']
x = cleanData[morningFeatures].copy()

In [24]:
x.columns

Index(['air_pressure_9am', 'air_temp_9am', 'avg_wind_direction_9am',
       'avg_wind_speed_9am', 'max_wind_direction_9am', 'max_wind_speed_9am',
       'rain_accumulation_9am', 'rain_duration_9am'],
      dtype='object')

In [25]:
y.columns

Index(['high_humidity_label'], dtype='object')

We now use the train_test_split function from sklearn which takes 2 arrays and returns for arrays that are subsets of the input arrays to split up our x,y data into random test and train subsets.

In [26]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.33, random_state = 324)

In [27]:
type(x_train)

pandas.core.frame.DataFrame

In [28]:
type(x_test)

pandas.core.frame.DataFrame

In [29]:
x_train.head()

,air_pressure_9am,air_temp_9am,avg_wind_direction_9am,avg_wind_speed_9am,max_wind_direction_9am,max_wind_speed_9am,rain_accumulation_9am,rain_duration_9am
841,918.370000,72.932000,184.500000,2.013246,186.700000,2.773806,0.0,0.0
75,920.100000,53.492000,186.100000,13.444009,193.800000,15.367778,0.0,0.0
95,927.610000,54.896000,55.000000,4.988376,53.400000,7.202947,0.0,0.0
895,919.235153,65.951112,194.343333,2.942019,216.569792,3.658810,0.0,0.0
699,919.888128,68.687822,228.517730,3.960858,247.954028,5.185547,0.0,0.0


In [30]:
y_train.describe()

,high_humidity_label
count,712.000000
mean,0.494382
std,0.500320
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


We now initialize a Decision Tree Classifier and then use the .fit() method to build a decision tree classifier from the training set.

In [31]:
humidityClassifier = DecisionTreeClassifier(max_leaf_nodes = 10, random_state = 0)

In [32]:
type(humidityClassifier)

sklearn.tree._classes.DecisionTreeClassifier

In [33]:
humidityClassifier.fit(x_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=10,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

We now use the DTC built from the training data to predict whether humidity is high or not.

In [34]:
predictions = humidityClassifier.predict(x_test)

In [35]:
type(predictions)

numpy.ndarray

In [36]:
predictions[0:10]

array([0, 0, 1, 1, 1, 1, 0, 0, 0, 1])

In [37]:
y_test['high_humidity_label'][0:10]

456     0
845     0
693     1
259     1
723     1
224     1
300     1
442     0
585     1
1057    1
Name: high_humidity_label, dtype: int32

We have now seen the first 10 values that were predicted by the model and the true values and then use the accuracy_score function to find the fraction of correct predictions.

In [38]:
accuracy_score(y_true = y_test, y_pred = predictions)

0.8153409090909091